## Import

In [1]:
!cp -r ../input/python-packages2 ./

!tar xvfz ./python-packages2/jiwer.tgz
!pip install ./jiwer/jiwer-2.3.0-py3-none-any.whl -f ./ --no-index
!tar xvfz ./python-packages2/normalizer.tgz
!pip install ./normalizer/bnunicodenormalizer-0.0.24.tar.gz -f ./ --no-index
!tar xvfz ./python-packages2/pyctcdecode.tgz
!pip install ./pyctcdecode/attrs-22.1.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/exceptiongroup-1.0.0rc9-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/hypothesis-6.54.4-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pygtrie-2.5.0.tar.gz -f ./ --no-index --no-deps
!pip install ./pyctcdecode/sortedcontainers-2.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pyctcdecode-0.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps

# !tar xvfz ./python-packages2/pypikenlm.tgz
# !pip install ./pypikenlm/pypi-kenlm-0.1.20220713.tar.gz -f ./ --no-index --no-deps

!cp -r /kaggle/input/pypi-kenlm/pypi-kenlm-0.1.20220713 ./
!pip install ./pypi-kenlm-0.1.20220713 --no-index --no-deps

jiwer/
jiwer/jiwer-2.3.0-py3-none-any.whl
jiwer/python-Levenshtein-0.12.2.tar.gz
jiwer/setuptools-65.3.0-py3-none-any.whl
Looking in links: ./
Processing ./jiwer/jiwer-2.3.0-py3-none-any.whl
INFO: pip is looking at multiple versions of jiwer to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement python-Levenshtein==0.12.2 (from jiwer) (from versions: none)
ERROR: No matching distribution found for python-Levenshtein==0.12.2
normalizer/
normalizer/bnunicodenormalizer-0.0.24.tar.gz
Looking in links: ./
Processing ./normalizer/bnunicodenormalizer-0.0.24.tar.gz
  Preparing metadata (setup.py) ... - done
  Created wheel for bnunicodenormalizer: filename=bnunicodenormalizer-0.0.24-py3-none-any.whl size=17628 sha256=ddfd5218084aadec7054172e8b4d70c3799f10fc0a5f03be155aaa7028aa5522
  Stored in directory: /root/.cache/pip/wheels/78/d7/75/6986dc3616718f950b80e3bd79a796ef618eaef6cd800e7909
Successf

In [2]:
!rm -r python-packages2 jiwer normalizer pyctcdecode pypikenlm
!rm -r pypi-kenlm-0.1.20220713

rm: cannot remove 'pypikenlm': No such file or directory


In [3]:
import typing as tp
from pathlib import Path
from functools import partial
from dataclasses import dataclass, field

import pandas as pd
import pyctcdecode
import numpy as np
from tqdm.notebook import tqdm

import librosa

import pyctcdecode
import kenlm
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor # , Wav2Vec2ProcessorWithLM
from bnunicodenormalizer import Normalizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
ROOT = Path.cwd().parent
INPUT = ROOT / "input"
DATA = INPUT / "bengaliai-speech"
TRAIN = DATA / "train_mp3s"
TEST = DATA / "test_mp3s"

SAMPLING_RATE = 16_000
MODEL_PATH = "/kaggle/input/facebook-1b-finetuned/best-hf-facebook-1b-funtune-stage3"
BEAM_WIDTH = 1024




# LM_PATH = Path("/kaggle/input/indiccorp-v1-v2-mix-lm-model")
LM_PATH = INPUT / "kenlm-custum-lm-model/mix_7gram_lm_model"
UNIGRAMS = LM_PATH/ "unigrams.txt"
BINARY = LM_PATH/ "7gram.bin"
CUSTUM_MODEL = "/kaggle/input/facebook-1b-finetuned/latest_hf_facebook_1b_fintune_stage_3_0921_dropout_0_8_noise_prob"
# CUSTUM_MODEL = "/kaggle/input/wav2vec-1b-v3/latest_hf_1013_12w_iters_stage3"

In [5]:
with open(str(UNIGRAMS)) as f:
    unigrams = f.readlines()

unigrams = [_.strip() for _ in unigrams if _.strip()]

In [6]:
len(unigrams)

6832852

### load model, processor, decoder

In [7]:
model = Wav2Vec2ForCTC.from_pretrained(CUSTUM_MODEL, torch_dtype=torch.float16)
processor = Wav2Vec2Processor.from_pretrained(MODEL_PATH)

In [8]:
vocab_dict = processor.tokenizer.get_vocab()
vocab_dict = vocab_dict["ben"]
vocab_dict['<s>'] = 64
vocab_dict['</s>'] =  65
sorted_vocab_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

decoder = pyctcdecode.build_ctcdecoder(
    list(sorted_vocab_dict.keys()),
    str(BINARY),
    unigrams=unigrams,
    alpha=0.375,
    beta=0.0625
)

## prepare dataloader

In [9]:
class BengaliSRTestDataset(torch.utils.data.Dataset):
    
    def __init__(
        self,
        audio_paths: list[str],
        sampling_rate: int
    ):
        self.audio_paths = audio_paths
        self.sampling_rate = sampling_rate
        
    def __len__(self,):
        return len(self.audio_paths)
    
    def __getitem__(self, index: int):
        audio_path = self.audio_paths[index]
        sr = self.sampling_rate
        w = librosa.load(audio_path, sr=sr)[0]
#         w = np.trim_zeros(w, "fb")
        return w

In [10]:
test = pd.read_csv(DATA / "sample_submission.csv", dtype={"id": str})
print(test.head())

             id                                           sentence
0  0f3dac00655e  এছাড়াও নিউজিল্যান্ড এ ক্রিকেট দলের হয়েও খেলছ...
1  a9395e01ad21  এছাড়াও নিউজিল্যান্ড এ ক্রিকেট দলের হয়েও খেলছ...
2  bf36ea8b718d  এছাড়াও নিউজিল্যান্ড এ ক্রিকেট দলের হয়েও খেলছ...


In [11]:
test_audio_paths = [str(TEST / f"{aid}.mp3") for aid in test["id"].values]

In [12]:
test_dataset = BengaliSRTestDataset(
    test_audio_paths, SAMPLING_RATE
)

collate_func = partial(
    processor.feature_extractor,
    # processor_with_lm.feature_extractor,
    return_tensors="pt", sampling_rate=SAMPLING_RATE,
    padding=True,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False,
    num_workers=1, collate_fn=collate_func, drop_last=False,
    pin_memory=True,
)

## Inference

In [13]:
if not torch.cuda.is_available():
    device = torch.device("cpu")
else:
    device = torch.device("cuda")
print(device)

cuda


In [14]:
model = model.to(device)
model = model.eval()

In [15]:
for batch in tqdm(test_loader):
    x = batch["input_values"]
    x = x.to(device).half()
    attn_mask = batch["attention_mask"]
    attn_mask = attn_mask.to(device)
    with torch.no_grad():
        y = model(input_values=x, attention_mask=attn_mask).logits
    y = y.detach().cpu().numpy()
    break

  0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
pred_sentence_list = []

for batch in tqdm(test_loader):
    x = batch["input_values"]
    x = x.to(device).half()
    attn_mask = batch["attention_mask"]
    attn_mask = attn_mask.to(device)
    with torch.no_grad():
        y = model(input_values=x, attention_mask=attn_mask).logits
    y = y.detach().cpu().numpy()
    attn_mask = attn_mask.sum(dim=1).cpu().numpy()
    for l, n in zip(y, attn_mask):
        valid_nums = int(n)
        beam = decoder.decode_beams(l[:valid_nums], beam_width=BEAM_WIDTH)
        s = beam[0][0]
        pred_sentence_list.append(s)

  0%|          | 0/3 [00:00<?, ?it/s]

## Make Submission

In [17]:
bnorm = Normalizer()

def postprocess(sentence):
    period_set = set([".", "?", "!", "।"])
    _words = [bnorm(word)['normalized']  for word in sentence.split()]
    sentence = " ".join([word for word in _words if word is not None])
    try:
        if sentence[-1] not in period_set:
            sentence+="।"
    except:
        # print(sentence)
        sentence = "।"
    return sentence

In [18]:
pp_pred_sentence_list = [
    postprocess(s) for s in tqdm(pred_sentence_list)]

  0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
test["sentence"] = pp_pred_sentence_list

test.to_csv("submission.csv", index=False)

print(test.head())

             id                                           sentence
0  0f3dac00655e                          একটু বয়স হলে একটি বিদেশি।
1  a9395e01ad21  কী কারণে তুমি এতাবৎ কাল পর্যন্ত এই দারুণ দৈবদু...
2  bf36ea8b718d  এ কারণে সরকার নির্ধারিত হারে পরিবহনজনিত ক্ষতি ...


## EOF

In [20]:
print(pp_pred_sentence_list)

['একটু বয়স হলে একটি বিদেশি।', 'কী কারণে তুমি এতাবৎ কাল পর্যন্ত এই দারুণ দৈবদুর্বিপাকে পতিত ছিলে বল।', 'এ কারণে সরকার নির্ধারিত হারে পরিবহনজনিত ক্ষতি অনুমোদন করে।']
